### Load preprocessed data

In [13]:
import numpy as np
fh = np.load('../data/dataset.npz')

# We have a bunch of feature columns and last column is the y-target
# Note pytorch is finicky about need int64 types
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']

# We've already split into train & test
test_x = fh['test_x'].astype(np.int64)
test_y = fh['test_y']


n_user = int(fh['n_user'])
n_item = int(fh['n_item'])

# columns are user_id, item_id and other features 
# we won't use the 3rd and 4th columns
print(train_x)
print(' ')
print(train_y)

[[   1 1193   11   10]
 [   1  914   26   10]
 [   1 3408    7   10]
 ...
 [6040  562   37    6]
 [6040 1096  109    6]
 [6040 1097   99    6]]
 
[[5.]
 [3.]
 [4.]
 ...
 [5.]
 [4.]
 [4.]]


### Define the MF Model

In [5]:
import torch
from torch import nn
import torch.nn.functional as F

In [14]:
class MF(nn.Module):
    itr = 0
    def __init__(self, n_user, n_item, k=18, c_vector=1.0, writer=None):
        super(MF, self).__init__()
        # This will hold the logging
        self.writer = writer
        
        # These are simple hyperparameters
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_vector = c_vector
        
        # These are learned and fit by PyTorch
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)
    
    def __call__(self, train_x):
        # This is the most import function in this script
        # These are the user indices, and correspond to "u" variable
        user_id = train_x[:, 0]
        # Item indices, correspond to the "i" variable
        item_id = train_x[:, 1]
        # vector user = p_u
        vector_user = self.user(user_id)
        # vector item = q_i
        vector_item = self.item(item_id)
        # this is a dot product & a user-item interaction: p_u * q_i
        ui_interaction = torch.sum(vector_user * vector_item, dim=1)
        return ui_interaction
    
    def loss(self, prediction, target):
        # MSE error between target = R_ui and prediction = p_u * q_i
        loss_mse = F.mse_loss(prediction, target.squeeze())
        # Compute L2 reularization over user (P) and item (Q) matrices
        prior_user =  l2_regularize(self.user.weight) * self.c_vector
        prior_item = l2_regularize(self.item.weight) * self.c_vector
        # Add up the MSE loss + user & item regularization
        total = loss_mse + prior_user + prior_item
        
        # This logs all local variables to tensorboard
        for name, var in locals().items():
            if type(var) is torch.Tensor and var.nelement() == 1 and self.writer is not None:
                self.writer.add_scalar(name, var, self.itr)
        return total

In [15]:
def l2_regularize(array):
    loss = torch.sum(array ** 2.0)
    return loss

### Train model

In [17]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss
from ignite.metrics import MeanSquaredError

from tensorboardX import SummaryWriter

from loader import Loader
from datetime import datetime

In [18]:
# Hyperparameters
lr = 1e-2
# Number of dimensions per user, item
k = 10
# regularization constant
c_vector = 1e-6

# Setup logging
log_dir = 'runs/simple_mf_01_' + str(datetime.now()).replace(' ', '_')
print(log_dir)
writer = SummaryWriter(log_dir=log_dir)

runs/simple_mf_01_2018-09-25_14:32:44.104497


In [19]:
model = MF(n_user, n_item, writer=writer, k=k, c_vector=c_vector)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

trainer = create_supervised_trainer(model, optimizer, model.loss)
metrics = {'accuracy': MeanSquaredError()}
evaluat = create_supervised_evaluator(model, metrics=metrics)

train_loader = Loader(train_x, train_y, batchsize=1024)
test_loader = Loader(test_x, test_y, batchsize=1024)

In [20]:
def log_training_loss(engine, log_interval=400):
    model.itr = engine.state.iteration
    if model.itr % log_interval == 0:
        fmt = "Epoch[{}] Iteration[{}/{}] Loss: {:.2f}"
        msg = fmt.format(engine.state.epoch, engine.state.iteration, 
                         len(train_loader), engine.state.output)
        print(msg)

trainer.add_event_handler(event_name=Events.ITERATION_COMPLETED, handler=log_training_loss)

def log_validation_results(engine):
    # When triggered, run the validation set
    evaluat.run(test_loader)
    avg_accuracy = evaluat.state.metrics['accuracy']
    print("Epoch[{}] Validation MSE: {:.2f} ".format(engine.state.epoch, avg_accuracy))
    writer.add_scalar("validation/avg_accuracy", avg_accuracy, engine.state.epoch)

trainer.add_event_handler(event_name=Events.EPOCH_COMPLETED, handler=log_validation_results)


model

MF(
  (user): Embedding(6041, 10)
  (item): Embedding(3953, 10)
)

#### Run model

In [21]:
trainer.run(train_loader, max_epochs=250)

Epoch[1] Iteration[400/879] Loss: 15.60
Epoch[1] Iteration[800/879] Loss: 4.64
Epoch[1] Validation MSE: 3.01 
Epoch[2] Iteration[1200/879] Loss: 1.28
Epoch[2] Iteration[1600/879] Loss: 1.10
Epoch[2] Validation MSE: 1.04 
Epoch[3] Iteration[2000/879] Loss: 0.93
Epoch[3] Iteration[2400/879] Loss: 1.03
Epoch[3] Validation MSE: 0.94 
Epoch[4] Iteration[2800/879] Loss: 0.89
Epoch[4] Iteration[3200/879] Loss: 0.88
Epoch[4] Validation MSE: 0.91 
Epoch[5] Iteration[3600/879] Loss: 0.89
Epoch[5] Iteration[4000/879] Loss: 0.88
Epoch[5] Validation MSE: 0.88 
Epoch[6] Iteration[4400/879] Loss: 0.82
Epoch[6] Iteration[4800/879] Loss: 0.79
Epoch[6] Iteration[5200/879] Loss: 0.88
Epoch[6] Validation MSE: 0.86 
Epoch[7] Iteration[5600/879] Loss: 0.87
Epoch[7] Iteration[6000/879] Loss: 0.84
Epoch[7] Validation MSE: 0.85 
Epoch[8] Iteration[6400/879] Loss: 0.75
Epoch[8] Iteration[6800/879] Loss: 0.86
Epoch[8] Validation MSE: 0.85 
Epoch[9] Iteration[7200/879] Loss: 0.76
Epoch[9] Iteration[7600/879] Loss

KeyboardInterrupt: 

In [23]:
model.user.weight.data.numpy()

array([[-1.48679995e-38,  2.67335097e-38, -1.46832621e-38, ...,
         1.76441049e-38, -1.73462281e-38,  2.27522386e-39],
       [ 2.52260685e-01, -4.76344436e-01, -7.33072221e-01, ...,
         1.02895653e+00,  3.48779172e-01,  9.55800772e-01],
       [ 5.46459198e-01, -8.06874454e-01, -7.48508647e-02, ...,
         9.29876029e-01,  8.31351280e-01,  1.39116681e+00],
       ...,
       [ 9.14424121e-01,  1.15741104e-01,  2.38355175e-01, ...,
         1.20365620e+00,  3.00456643e-01,  1.17352653e+00],
       [ 3.32125604e-01, -1.64092541e-01, -1.85646057e-01, ...,
         1.12085617e+00, -6.29628822e-02,  1.14474487e+00],
       [ 1.41553831e+00, -3.88202786e-01, -7.55467653e-01, ...,
         8.46764863e-01, -2.44182423e-01,  7.36870408e-01]], dtype=float32)